In [106]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [107]:
df = pd.read_csv("results.csv")

In [108]:
df

,INDICATOR,Unemployment actual,INDICATOR.1,RealGDP actual,INDICATOR.2,Core CPI actual,INDICATOR.3,Core PCE actual,XGBoost Unemployment,XGBoost RealGDP,XGBoost Core CPI,XGBoost Core PCE,Random Forest Unemployment,Random Forest RealGDP,Random Forest Core CPI,Random Forest Core PCE,Linear Regression Unemployment,Linear Regression RealGDP,Linear Regression Core CPI,Linear Regression Core PCE,KNeigborsRegressor Unemployment,KNeigborsRegressor RealGDP,KNeigborsRegressor Core CPI,KNeigborsRegressor Core PCE
0,Unemployment,0.000000,RealGDP,2.99646,Core CPI,2.290573,Core PCE,1.184167,0.081926,2.500126,2.291859,1.232153,0.081083,2.609857,2.279672,1.250697,2.763758,1.913119,2.037117,2.153685,1.621667,2.563892,2.270754,1.237391
1,Unemployment,3.891667,RealGDP,4.44722,Core CPI,2.174324,Core PCE,1.184167,3.820591,4.123776,2.132030,1.245136,3.866667,4.211068,2.174324,1.404765,4.956422,2.258982,1.673650,1.083818,3.850000,4.319114,2.174324,1.439257
2,Unemployment,3.683333,RealGDP,4.75324,Core CPI,2.174324,Core PCE,1.647664,3.852198,4.178174,2.079864,1.627884,3.716916,4.146253,1.940770,1.644099,3.938026,2.474587,1.802591,1.909056,3.808333,4.387366,1.562823,1.647664
3,Unemployment,8.108333,RealGDP,2.99646,Core CPI,2.290573,Core PCE,1.664742,8.142979,1.312183,2.193956,1.751450,8.108333,0.570160,2.223187,1.726642,4.816343,0.651697,2.003969,2.245712,6.338333,-1.492464,2.231115,1.727602
4,Unemployment,4.350000,RealGDP,4.75324,Core CPI,2.290573,Core PCE,1.559324,4.346418,4.914411,2.220703,1.585213,4.350000,4.559254,2.222196,1.718909,4.817755,4.013551,2.108200,2.264820,4.350000,4.377784,2.249111,1.975634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2665,NaN,NaN,RealGDP,2.68428,NaN,NaN,NaN,NaN,NaN,2.994133,NaN,NaN,NaN,2.939068,NaN,NaN,NaN,4.166291,NaN,NaN,NaN,3.759920,NaN,NaN
2666,NaN,NaN,RealGDP,3.07551,NaN,NaN,NaN,NaN,NaN,2.578147,NaN,NaN,NaN,2.686900,NaN,NaN,NaN,1.719996,NaN,NaN,NaN,2.108490,NaN,NaN
2667,NaN,NaN,RealGDP,4.12748,NaN,NaN,NaN,NaN,NaN,4.376538,NaN,NaN,NaN,4.127480,NaN,NaN,NaN,3.387583,NaN,NaN,NaN,4.252632,NaN,NaN
2668,NaN,NaN,RealGDP,2.56377,NaN,NaN,NaN,NaN,NaN,2.506146,NaN,NaN,NaN,2.401250,NaN,NaN,NaN,2.320658,NaN,NaN,NaN,2.375238,NaN,NaN


In [109]:
uX = df[["XGBoost Unemployment", "Random Forest Unemployment", "Linear Regression Unemployment", "KNeigborsRegressor Unemployment"]]
uY = df[["Unemployment actual"]]

uX = uX.dropna()
uY = uY.dropna()

In [110]:
gX = df[["XGBoost RealGDP", "Random Forest RealGDP", "Linear Regression RealGDP", "KNeigborsRegressor RealGDP"]]
gY = df[["RealGDP actual"]]

In [111]:
cX = df[["XGBoost Core CPI", "Random Forest Core CPI", "Linear Regression Core CPI", "KNeigborsRegressor Core CPI"]]
cY = df[["Core CPI actual"]]

cX = cX.dropna()
cY = cY.dropna()

In [112]:
pX = df[["XGBoost Core PCE", "Random Forest Core PCE", "Linear Regression Core PCE", "KNeigborsRegressor Core PCE"]]
pY = df[["Core PCE actual"]]

pX = pX.dropna()
pY = pY.dropna()

In [113]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [114]:
def create_mlp(dim, regress=False):
	# define our MLP network
	model = Sequential()
	model.add(Dense(8, input_dim=dim, activation="relu"))
	model.add(Dense(4, activation="relu"))
	# check to see if the regression node should be added
	if regress:
		model.add(Dense(1, activation="linear"))
	# return our model
	return model

In [115]:
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import numpy as np
import argparse
import locale
import os

model = create_mlp(uX.shape[1], regress=True)
opt = Adam(lr=1e-3, decay=1e-3 / 200)
model.compile(loss="mean_absolute_percentage_error", optimizer=opt)
# train the model
print("[INFO] training model...")
model.fit(x=uX, y=uY, 
	validation_data=(uX, uY),
	epochs=200, batch_size=8)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


[INFO] training model...
Epoch 1/200
200/200 [==============================] - 1s 4ms/step - loss: 6997378.2114 - val_loss: 1221735.7500
Epoch 2/200
200/200 [==============================] - 0s 2ms/step - loss: 809068.8258 - val_loss: 544189.2500
Epoch 3/200
200/200 [==============================] - 0s 2ms/step - loss: 275275.5538 - val_loss: 330439.0312
Epoch 4/200
200/200 [==============================] - 1s 3ms/step - loss: 257432.1025 - val_loss: 86610.5078
Epoch 5/200
200/200 [==============================] - 0s 2ms/step - loss: 78288.7826 - val_loss: 158498.6562
Epoch 6/200
200/200 [==============================] - 0s 2ms/step - loss: 68911.9352 - val_loss: 157228.6562
Epoch 7/200
200/200 [==============================] - 0s 2ms/step - loss: 111881.8609 - val_loss: 150679.5312
Epoch 8/200
200/200 [==============================] - 0s 2ms/step - loss: 186228.0078 - val_loss: 67666.3672
Epoch 9/200
200/200 [==============================] - 0s 2ms/step - loss: 55912.9758 - v

In [116]:
preds = model.predict(uX)

In [117]:
preds

array([[-5.636699e-05],
       [-5.636699e-05],
       [-5.636699e-05],
       ...,
       [-5.636699e-05],
       [-5.636699e-05],
       [-5.636699e-05]], dtype=float32)

In [118]:
from sklearn.linear_model import LinearRegression

In [119]:
reg = LinearRegression().fit(gX, gY)
reg.score(gX, gY)

0.9549310708629474

In [120]:
reg.predict(gX)

array([[2.56188736],
       [4.1774761 ],
       [4.15642966],
       ...,
       [4.21928957],
       [2.43410261],
       [4.23721564]])

In [121]:
reg.coef_

array([[ 0.39451943,  0.62355299, -0.00584316, -0.0121875 ]])

In [122]:
from sklearn.metrics import mean_squared_error

pred = reg.predict(gX)
actual = gY

np.sqrt(mean_squared_error(pred, actual))

0.4089700273137137

In [123]:
reg = LinearRegression().fit(gX, gY)
print(reg.score(gX, gY))

pred = reg.predict(gX)
actual = gY

np.sqrt(mean_squared_error(pred, actual))

0.9549310708629474


0.4089700273137137

In [124]:
reg = LinearRegression().fit(uX, uY)
print(reg.score(uX, uY))

pred = reg.predict(uX)
actual = uY

np.sqrt(mean_squared_error(pred, actual))

0.9437394066880166


0.6735477307884534

In [125]:
reg = LinearRegression().fit(cX, cY)
print(reg.score(cX, cY))

pred = reg.predict(cX)
actual = cY

np.sqrt(mean_squared_error(pred, actual))

0.9251698478091955


0.33160928669688405

In [126]:
reg = LinearRegression().fit(pX, pY)
print(reg.score(pX, pY))

pred = reg.predict(pX)
actual = pY

np.sqrt(mean_squared_error(pred, actual))

0.9311808648943892


0.31517548684660446